In [27]:
#Import Libraries
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
#%pip install cloudscraper
#import cloudscraper
#%pip install selenium
import selenium
import random
#%pip install undetected-chromedriver
#import undetected_chromedriver.v2 as uc
#%pip install selenium-stealth



In [19]:
# Get_page_reviews function scrapes all reviews, titles, and scores
# and returns a data frame containing the results across 3 columns

def get_page_reviews(soup):
    review_df = pd.DataFrame(columns = ["title","review","score"])
    reviews = soup.find_all("div", {"class": "mb-0_wJE overflow-hidden_1pr card_12c card-full_1bA card-full-md_1Kd"})
    
    for review in reviews:
        score = review.find_all("div", {"class":  re.compile('me-1_2lk flex-shrink-0_2yX.*')})
        score = re.search('title="(\d)\sout\sof',str(score)).group(1)

        title = review.find_all("h3", {"class":  "mb-1_2Ff"})
        title = re.search('>(.+?)<',str(title)).group(1)

        reviews = review.find_all("div", {"class":  "relative_1Fn text-break_iZk"})
        for review_segment in reviews:
            review_start = review_segment.find('p')
            review_end = review_segment.find("span", {"class":  "d-none_21M"})

            if review_end is not None:
                review_complete = re.sub("\.\.\.$", "", review_start.text) + re.sub("^\.\.\.", "", review_end.text)
            else:
                review_complete = review_start.text

            review_df = review_df.append(pd.DataFrame([[title, review_complete, score]], columns = ["title","review","score"]))
            
    return review_df

    

In [20]:
# Create empty dataframe for vodafone reviews
vodafone_reviews = pd.DataFrame(columns = ["title","review","score"])

vodafone_reviews = pd.read_csv('vodafone_reviews.csv')  

In [31]:


# use selenium cloudscraper to avoid bot detection
from selenium import webdriver
from selenium_stealth import stealth

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

# options.add_argument("--headless")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get('https://www.productreview.com.au/listings/vodafone?page=10')
#driver.get('https://bot.sannysoft.com')
time.sleep(60)
page = driver.page_source

#Parse the HTML into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, 'html.parser')


review_page = 10

# loop through all vodafone review pages, and add reviews to vodafone_review dataframe
while 1==1:
    
    # get reviews on current page
    cur_reviews = get_page_reviews(soup)
    
    # break if no reviews
    if len(cur_reviews)==0:
        break

    # Append reviews for current page
    vodafone_reviews = vodafone_reviews.append(cur_reviews)
 
    # Get link for next page
  
    review_page += 1
    
    next_page = "https://www.productreview.com.au/listings/vodafone?page=" + str(review_page)
    print(next_page)
    
    # load  and parse the next page of reviews
    driver.get(next_page)
    time.sleep(120 + random.randint(0,30))
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')


WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.164)


In [ ]:
print(len(vodafone_reviews))

In [ ]:
vodafone_reviews.to_csv ('vodafone_reviews.csv', index = None, header=True)

In [48]:
##########################################
##  Some EDA

def strip_punct(word):
    word =  re.sub("[,\.;:\(\)\{\}\[\]]", " ", word)
    word =  re.sub("\s+", " ", word)
    return word

def list_of_words(df, column):
    word_list = []
    for i in range(len(df)):
        words = strip_punct(df[column].iloc[i])
        words = words.split(" ")
        word_list = word_list + words
        
    return word_list

word_list = list_of_words(vodafone_reviews, "title")

word_list.value_counts(sort=True)

AttributeError: 'list' object has no attribute 'value_counts'

In [ ]:
list_of_words